In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,11174
2,Huelva,Confirmados PDIA 14 días,828
3,Huelva,Tasa PDIA 14 días,"162,11675930947658"
4,Huelva,Confirmados PDIA 7 días,276
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,5
629,Municipio de Huelva sin especificar,Total Confirmados,129
630,Municipio de Huelva sin especificar,Curados,113


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  11174.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3155.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1916 personas en los últimos 7 días 

Un positivo PCR cada 611 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,11174.0,276.0,828.0,510743.0,54.038920,162.116759,51.0
Huelva-Costa,6316.0,170.0,538.0,288115.0,59.004217,186.730993,30.0
Condado-Campiña,3443.0,97.0,262.0,155057.0,62.557640,168.970121,17.0
Huelva (capital),3155.0,75.0,235.0,143663.0,52.205509,163.577261,10.0
Isla Cristina,457.0,27.0,74.0,21264.0,126.975169,348.006020,7.0
Palos de la Frontera,257.0,17.0,36.0,11289.0,150.589069,318.894499,6.0
Lepe,975.0,21.0,65.0,27431.0,76.555722,236.958186,5.0
Moguer,385.0,15.0,45.0,22088.0,67.910177,203.730532,3.0
Cartaya,426.0,27.0,75.0,19974.0,135.175728,375.488135,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Lucena del Puerto,105.0,6.0,22.0,3371.0,177.988727,652.625334,1.0
Palma del Condado (La),547.0,20.0,62.0,10761.0,185.856333,576.154632,2.0
Cartaya,426.0,27.0,75.0,19974.0,135.175728,375.488135,3.0
Bonares,67.0,11.0,22.0,6058.0,181.578079,363.156157,1.0
Isla Cristina,457.0,27.0,74.0,21264.0,126.975169,348.006020,7.0
Palos de la Frontera,257.0,17.0,36.0,11289.0,150.589069,318.894499,6.0
Lepe,975.0,21.0,65.0,27431.0,76.555722,236.958186,5.0
Aljaraque,440.0,8.0,46.0,21260.0,37.629351,216.368768,1.0
Moguer,385.0,15.0,45.0,22088.0,67.910177,203.730532,3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Ayamonte,337.0,3.0,20.0,20946.0,14.322544,95.483625,1.0,0.150000
Aljaraque,440.0,8.0,46.0,21260.0,37.629351,216.368768,1.0,0.173913
Bollullos Par del Condado,251.0,3.0,16.0,14272.0,21.020179,112.107623,1.0,0.187500
Sierra de Huelva-Andévalo Central,1287.0,4.0,21.0,67571.0,5.919699,31.078421,1.0,0.190476
San Juan del Puerto,154.0,2.0,10.0,9300.0,21.505376,107.526882,0.0,0.200000
Punta Umbría,236.0,2.0,9.0,15242.0,13.121638,59.047369,1.0,0.222222
Lucena del Puerto,105.0,6.0,22.0,3371.0,177.988727,652.625334,1.0,0.272727
Huelva-Costa,6316.0,170.0,538.0,288115.0,59.004217,186.730993,30.0,0.315985
Huelva (capital),3155.0,75.0,235.0,143663.0,52.205509,163.577261,10.0,0.319149
